### Requires bbox.csv from bbox_maker.ipynb

In [2]:
from os.path import exists
from time import sleep
import requests
import pandas as pd
import numpy as np
from collections import defaultdict
import ast
from json import dump, load
from IPython.display import clear_output

In [3]:
START = 6400
END = 6600
DAILY_MAX = 500
BUFFER = 2000

In [4]:
# Read in API keys
with open('../data/raw/api.json', 'r') as f:
  keys = load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/api.json'

In [5]:
key_vals = list(keys.values())
num_keys = len(key_vals)

NameError: name 'keys' is not defined

In [6]:
# Read in coordinate data
data = pd.read_csv('../data/curated/bbox.csv', index_col=0)

In [7]:
data

,coordinates,lat,long,bbox_min_lat,bbox_min_long,bbox_max_lat,bbox_max_long
0,"[-36.253557, 144.944557]",-36.253557,144.944557,-36.433421,144.721514,-36.073693,145.167600
1,"[-36.554159, 146.7208003]",-36.554159,146.720800,-36.734023,146.496892,-36.374295,146.944708
2,"[-36.5530096, 146.7188613]",-36.553010,146.718861,-36.732874,146.494957,-36.373146,146.942765
3,"[-36.5380049, 145.9927626]",-36.538005,145.992763,-36.717869,145.768902,-36.358141,146.216624
4,"[-36.5424309, 145.9979175]",-36.542431,145.997918,-36.722295,145.774044,-36.362567,146.221792
...,...,...,...,...,...,...,...
6595,"[-37.8223967, 144.938237]",-37.822397,144.938237,-38.002261,144.710537,-37.642533,145.165937
6596,"[-37.8301164, 144.9569041]",-37.830116,144.956904,-38.009980,144.729180,-37.650252,145.184628
6597,"[-37.8141725, 144.9740049]",-37.814172,144.974005,-37.994036,144.746330,-37.634308,145.201680
6598,"[-38.372703, 144.7856897]",-38.372703,144.785690,-38.552567,144.556269,-38.192839,145.015111


In [8]:
# Check if .json file already exists, and create file if not
if not exists(f'../data/curated/POIs/POIs/poi_{START+1}_{END}.json'):
    with open(f'../data/curated/POIs/POIs/poi_{START+1}_{END}.json', 'w') as f:
        dump([], f)

In [26]:
# Query from OpenPOIService to obtain a list of POIs in a 2km range around each property
# Rotate through API keys to eliminate risk of query limit
for i in range(START, END):
    data_dict = defaultdict(dict)
    feature_list = []
    
    body = {"request":"pois","geometry":{"geojson":{"type":"Point","coordinates":[data.long[i], data.lat[i]]},"buffer":BUFFER}}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': f'{key_vals[i%num_keys]}',
        'Content-Type': 'application/json; charset=utf-8'
    }
    
    clear_output(wait=True)
    print(f'Query number {i}:')
    call = requests.post('https://api.openrouteservice.org/pois', json=body, headers=headers)
    print(f'Code {call.status_code}, {call.reason}')
    temp = ast.literal_eval(call.__dict__['_content'].decode('utf-8'))
    
    try:
        features = temp['features']
        #data_dict[data.coordinates[0]]['features'][0]['properties']['category_ids'].values()
        for elem in features:
            coords = elem['geometry']['coordinates']
            dist = elem['properties']['distance']
            category = list(elem['properties']['category_ids'].values())[0]
            feature_list.append([dict({'feature_coordinates': coords}), dict({'distance': dist}), category])
        
        data_dict[data.coordinates[i]] = feature_list
    except:
        category = dict({"category_name": "", "category_group": ""})
        data_dict[data.coordinates[i]] = [dict({'feature_coordinates': np.nan}), dict({'distance': np.nan}), category]
    
    
    with open(f'../data/curated/POIs/POIs/poi_{START+1}_{END}.json', 'r') as f:
        file = load(f)
    file.append(data_dict)
    with open(f'../data/curated/POIs/POIs/poi_{START+1}_{END}.json', 'w') as f:
        dump(file, f, indent=4, separators=(',',': '))

Query number 6599:
Code 200, OK


In [3]:
dict({'coords': [dict({'feature_coordinates': np.nan}), dict({'distance': np.nan}), dict({"category_name": "", "category_group": ""})]})

{'coords': [{'feature_coordinates': nan},
  {'distance': nan},
  {'category_name': '', 'category_group': ''}]}

In [5]:
with open(f'../data/curated/POIs/POIs/poi_{START+1}_{END}.json', 'r') as f:
        file = load(f)
len(file)

200

In [6]:
file[100]

{'[-37.8477661, 144.9786951]': [[{'feature_coordinates': [144.981557,
     -37.841123]},
   {'distance': 779.1211317},
   {'category_name': 'park', 'category_group': 'leisure_and_entertainment'}],
  [{'feature_coordinates': [144.970918, -37.845823]},
   {'distance': 717.6443548},
   {'category_name': 'park', 'category_group': 'leisure_and_entertainment'}],
  [{'feature_coordinates': [144.972236, -37.842331]},
   {'distance': 828.94017284},
   {'category_name': 'golf_course',
    'category_group': 'leisure_and_entertainment'}],
  [{'feature_coordinates': [144.961767, -37.842802]},
   {'distance': 1588.56954524},
   {'category_name': 'sports_centre',
    'category_group': 'leisure_and_entertainment'}],
  [{'feature_coordinates': [144.963713, -37.841425]},
   {'distance': 1494.77926167},
   {'category_name': 'parking', 'category_group': 'transport'}],
  [{'feature_coordinates': [144.965317, -37.840485]},
   {'distance': 1428.15484999},
   {'category_name': 'stadium',
    'category_group':

In [20]:
list(file[100].values())[0][2][2]

{'category_name': 'golf_course', 'category_group': 'leisure_and_entertainment'}